# Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector
import json
from tqdm import tqdm

# Import data

In [2]:
# Read offers_canarias.json as a DataFrame in the folder data
df = pd.read_json('data/offers_canarias.json', encoding='utf-8')
# Add the column 'salaryOriginal' with NaN values
df['salaryOriginal'] = np.nan
# Define a custom function to access the 'label' key in a dictionary
def get_label(d):
    return d.get('label')
# Apply the custom function to the 'location' and 'category' columns
df['location'] = df['location'].apply(get_label)
df['category'] = df['category'].apply(get_label)
print(df.columns)
print(df.shape)
df.head()

Index(['description', 'postings', 'company', 'location', 'category', 'jobType',
       'title', 'numberOfVacancies', 'salaryOriginal'],
      dtype='object')
(156676, 9)


,description,postings,company,location,category,jobType,title,numberOfVacancies,salaryOriginal
0,Búsqueda y captación de clientes para la venta...,"[{'date': '2021-01-01', 'site': {'label': 'gla...",MGS Seguros y Reaseguros,Santa Cruz de Tenerife,Finanzas - Economía,Indefinido,Asesor Profesional de Seguros,NaN,NaN
1,¿Quieres unirte a nosotros y trabajar en un en...,"[{'date': '2021-03-19', 'site': {'label': 'es....",Grupo Eulen,Las Palmas de Gran Canaria,Medicina - Salud,Indefinido,Enfermería,NaN,NaN
2,Si está buscando trabajo en el sector inmobili...,"[{'date': '2021-01-01', 'site': {'label': 'gla...",Century 21,Santa Cruz de Tenerife,Comercial - Ventas,Indefinido,Asesor Inmobiliario,NaN,NaN
3,"Buscamos un/a apasionad@ del ciclismo, que le ...","[{'date': '2021-07-16', 'site': {'label': 'gla...",Decathlon España,La Laguna,Retail,Tiempo Parcial,VENDEDOR DEPORTISTA DEL CICLISMO Decathlon La ...,NaN,NaN
4,Descripción:\n¡Ampliamos nuestro equipo técnic...,"[{'date': '2021-01-01', 'site': {'label': 'gla...",Oesia - Oesía Networks,Canarias,IT - Sistemas - Tecnología,Indefinido,Analista Senior de Ciberseguridad,8.0,NaN


# Test connection to database

In [3]:
# Test the connection to the database
config = {
    'user': 'root',
    'password': 'root',
    'host': 'localhost',
    'database': 'test',
    'port': '3306'
}

# SELECT * FROM test.jobmarket_canarias;
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()
cursor.execute('SELECT * FROM test.ofertas_jobmarket_canarias_21_23')
print(cursor.fetchall())

cursor.execute('SELECT * FROM test.postings_jobmarket_canarias_21_23')
print(cursor.fetchall())

[(21188, 'Búsqueda y captación de clientes para la venta de seguros. Realización de labores administrativa y gestión de siniestros.\nTipo de puesto: Jornada completa\nSalario: 13.968,00€-18.000,00€ al año\nExperiencia:\nComercial: 1 año (Requisito deseable)\nUbicación:\nSanta Cruz de Tenerife, Santa Cruz de Tenerife provincia (Requisito mínimo)\nFecha de incorporación:\nInmediatamente (Requisito deseable)\nComercial:\nDe 1 a 2 años (Requisito deseable)\nTareas del puesto:\nAdministrar varias cuentas, asegurando que se mantenga un nivel alto de retención y satisfacción del cliente\nCumplir con las cuotas mensuales\nTratar con múltiples clientes a diario, generando nuevos clientes y desarrollando campañas para mantener la lealtad de los mismos\nImplementar estrategias para retener a los clientes que soliciten cancelar o reducir su cuenta\nTrabajar desde casa:\nSí', 'Asesor Profesional de Seguros', 'MGS Seguros y Reaseguros', 'Santa Cruz de Tenerife', 'Finanzas - Economía', 'Indefinido', 

# Describe table

In [4]:
# Describe the table
cursor.execute('DESCRIBE test.ofertas_jobmarket_canarias_21_23')
# Print the result
describe = cursor.fetchall()
# Print the column names
columns = [column[0] for column in describe]

# Import data to database

In [8]:
# Create a list to store the values you want to insert into the postings table
postings_data = []
# Iterate over the rows in the DataFrame
for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    # Create a list of dictionaries with the postings
    postings = row['postings']
    # Create an object with all the columns except "postings"
    row_data = row.drop('postings')
    row_data = row_data.to_dict()
    # Replace NaN values with None
    for key, value in row_data.items():
        if pd.isna(value):
            row_data[key] = None
    # Serialize the dictionary to a JSON string
    row_data_json = json.dumps(row_data)
    # Insert into the ofertas_jobmarket_canarias_21_23 table and get the id
    try:
        cursor.execute('INSERT INTO test.ofertas_jobmarket_canarias_21_23 ({}) VALUES ({})'.format(', '.join(row_data.keys()), ', '.join(['%s'] * len(row_data))), list(row_data.values()))
    except mysql.connector.errors.IntegrityError:
        continue
    id = cursor.lastrowid
    # Create a for loop over the postings list
    for posting in postings:
        # Rename the actual id in posting to id_posting
        posting['id_posting'] = posting.pop('id')
        # Add the id of the ofertas_jobmarket_canarias_21_23 table to the posting
        posting['id'] = id
        # Verify if 'salaryOriginal' is in the posting
        if 'salaryOriginal' in posting:
            # Then update the record inserted in ofertas_jobmarket_canarias_21_23 and add the salaryOriginal
            cursor.execute('UPDATE test.ofertas_jobmarket_canarias_21_23 SET salaryOriginal = %s WHERE id = %s', (posting['salaryOriginal'], id))
        # Replace NaN values with None in posting
        for key, value in posting.items():
            if pd.isna(value):
                posting[key] = None
        # Append the posting data to the list
        postings_data.append(posting)
    cnx.commit()
# Iterate over the postings_data list with a tqdm progress bar
for posting in tqdm(postings_data, desc="Inserting postings"):
    try:
        cursor.execute('INSERT INTO test.postings_jobmarket_canarias_21_23 ({}) VALUES ({})'.format(', '.join(posting.keys()), ', '.join(['%s'] * len(posting))), list(posting.values()))
    except mysql.connector.errors.IntegrityError:
        continue
    cnx.commit()

KeyboardInterrupt: 